In [1]:
import tkinter as tk
import time
import random
import sqlite3
from threading import Thread
import pygame
import serial
pygame.mixer.init()
from tkinter import PhotoImage
from PIL import Image, ImageTk

class Game:
    def __init__(self, team_name, leaderboard):
        self.team_name = team_name
        self.leaderboard = leaderboard
        self.score = 0

        # Add these lines to open the serial port
        self.ser = serial.Serial()
        self.ser.baudrate = 115200
        self.ser.port = 'COM1'  # Change this to the appropriate serial port
        self.ser.open()

        self.hit_sound = pygame.mixer.Sound("tie_hit.mp3")
        self.miss_sound = pygame.mixer.Sound("death_miss.mp3")
        self.tunnel_sound = pygame.mixer.Sound("seismic_tunnel.mp3")
        
        self.hit_color = "lightgreen"
        self.miss_color = "pink"  
        self.tunnel_color = "purple"  

        startup_sound = pygame.mixer.Sound("tie_start.mp3")
        startup_sound.play()

    def simulate(self, update_callback, leaderboard_callback):
        tries = 0
        while tries < 15:
            if self.ser.in_waiting:
                data = (self.ser.readline().decode().strip())
                self.update_score(data, update_callback)
                tries += 1

                if tries % 15 == 0:
                    self.update_leaderboard(leaderboard_callback)

    def update_score(self, data, callback):
        if data == "Hit":
            self.score += 1
            self.hit_sound.play()
            callback(f" HIT!!!!!!   Score:   {self.score}", self.hit_color)
        
        if data == "Hit & Miss":
            self.score += 3
            self.tunnel_sound.play()
            callback(f" TUNNEL!!!!!!   Score:  {self.score}", self.tunnel_color)
            
        
        
        if data == "Miss":
            self.miss_sound.play()
            callback(f" MISS!!!   Score:   {self.score}", self.miss_color)

    def update_leaderboard(self, callback):
        self.leaderboard.append((self.team_name.get(), self.score))
        self.leaderboard.sort(key=lambda x: x[1], reverse=True)
        callback(self.leaderboard)

        # Add this line to close the serial port when the game is over
        self.ser.close()

# The rest of the code remains the same, starting from the ScoreboardGUI class definition

class ScoreboardGUI:
    def __init__(self, master):
        

        self.master = master
        self.master.title("Scoreboard")
        
         # Load the background image as a PIL.Image object
       # self.background_image = Image.open("cats2.png")

        

        
        # Load the background image
        self.background_image = PhotoImage(file="new2.png")
        
        
       
     #   .wm_attributes('-transparentcolor','black')
     
        # Set the background image as the background of the GUI
        self.background_label = tk.Label(self.master, image=self.background_image)
        self.background_label.place(x=0, y=0, relwidth=1, relheight=1)
       # self.background_label.wm_attributes('-alpha', 0.7)
        
        
       
        #self.master.configure(bg= "black")
        
        # Set the window size to full-screen
        self.master.attributes("-fullscreen", True)

        # Set the window size to 768 pixels in height
        screen_height = self.master.winfo_screenheight()
        self.master.geometry(f"{screen_height}x{screen_height}")
        
        

        self.team_name = tk.StringVar()
       # tk.Label(self.master, text="team name:", font=("Gothic", 15), bg="black", fg="yellow").grid(row=1, column=2, pady=20)
        self.team_entry = tk.Entry(self.master, textvariable=self.team_name)
        self.team_entry.grid(row=1, column=3, pady=20)

        self.start_button = tk.Button(self.master, text="Start", command=self.start_game)
        self.start_button.grid(row=1, column=3, pady=30)


      #  self.leaderboard_label = tk.Label(self.master, text="", font=("Gothic", 18), bg="black", fg="yellow")
       # self.leaderboard_label.grid(row=0, column=0, rowspan=2, padx=50)
       # self.leaderboard_label.configure(font=("Gothic", 14))
        
        # Place the leaderboard label on the left-hand side of the screen
        self.leaderboard_label = tk.Label(self.master, text="", font=("Gothic", 18), bg="black", fg="yellow")
        self.leaderboard_label.grid(row=0, column=0, rowspan=2, padx=50, sticky=tk.W)
        self.leaderboard_label.configure(font=("Gothic", 20))
        
         # Center the scoreboard label
        self.scoreboard_frame = tk.Frame(self.master, bg="black")
        self.scoreboard_frame.place(relx=0.5, rely=0.3, anchor=tk.CENTER)
        
        self.score_label = tk.Label(self.scoreboard_frame, text="Score: 0", font=("Gothic", 36), bg="black", fg="yellow")
        self.score_label.grid(row=0, column=0, pady=20)
        self.color_label = tk.Label(self.scoreboard_frame, bg="white", width=100, height=30)
        self.color_label.grid(row=1, column=0)
        
        #tk.Label(self.master, text="team name:", font=("Gothic", 15), bg="black", fg="yellow").grid(row=0, column=4, pady=20)
        
        self.team_entry.place(relx=0.9, rely=0.1, anchor=tk.NE)
        self.start_button.place(relx=0.95, rely=0.1, anchor=tk.NE)
    
    def start_game(self):
        if not self.team_name.get():
            return
        
        
        
        self.game = Game(self.team_name, self.load_leaderboard())
        Thread(target=self.game.simulate, args=(self.update_scoreboard, self.update_leaderboard)).start()

    def load_leaderboard(self):
        conn = sqlite3.connect('leaderboard.db')
        c = conn.cursor()
        c.execute('''CREATE TABLE IF NOT EXISTS leaderboard
                     (team text, score integer)''')
        conn.commit()
        c.execute('SELECT * FROM leaderboard ORDER BY score DESC')
        leaderboard = [((row[0], row[1])) for row in c.fetchall()]
        conn.close()
        return leaderboard
    
    def save_leaderboard(self, leaderboard):
        conn = sqlite3.connect('leaderboard.db')
        c = conn.cursor()
        c.execute('DELETE FROM leaderboard')
        c.executemany('INSERT INTO leaderboard VALUES (?, ?)', leaderboard)
        conn.commit()
        conn.close()

    def update_scoreboard(self, text, color):
        self.score_label.configure(text=text, fg=color)
        self.color_label.configure(bg=color)
        
        self.master.after(500, lambda: self.score_label.configure(bg="black"))
        self.master.after(500, lambda: self.color_label.configure(bg="white"))
       # self.master.after(500, lambda: self.color_label.destroy)

    def update_leaderboard(self, leaderboard):
        leaderboard_text = "\n".join([f"{i+1}. {team} - {score}" for i, (team, score) in enumerate(leaderboard)])
        self.leaderboard_label.configure(text=leaderboard_text)
        self.save_leaderboard(leaderboard)
        
    

if __name__ == "__main__":
    pygame.mixer.music.load("tie.mp3")
    pygame.mixer.music.play(loops=-1)
    window = tk.Tk()
    ScoreboardGUI(window)
    window.config(background="black")
    window.title("Quantum wars")
    window.mainloop()



pygame 2.1.2 (SDL 2.0.18, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


Exception in thread Thread-5:
Traceback (most recent call last):
  File "C:\Users\Ben\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\Ben\anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Ben\AppData\Local\Temp\ipykernel_7860\657495193.py", line 40, in simulate
  File "C:\Users\Ben\AppData\Local\Temp\ipykernel_7860\657495193.py", line 61, in update_score
  File "C:\Users\Ben\AppData\Local\Temp\ipykernel_7860\657495193.py", line 172, in update_scoreboard
  File "C:\Users\Ben\anaconda3\lib\tkinter\__init__.py", line 1646, in configure
    return self._configure('configure', cnf, kw)
  File "C:\Users\Ben\anaconda3\lib\tkinter\__init__.py", line 1636, in _configure
    self.tk.call(_flatten((self._w, cmd)) + self._options(cnf))
RuntimeError: main thread is not in main loop
Exception ignored in: <function Image.__del__ at 0x0000023405A26A60>
Traceback (most recent call last):
  File "C:\Use